# Import potrebných knižníc

In [ ]:
import os
from datetime import datetime
from PIL import Image
import numpy as np

# Spájanie obrázkov BU

Spájanie obrázkov prebieha na intervale 30 minút

Tento krok je vykonávaný pretože k jednému snímku GOLD by pripadalo viacero snímok BU

Po konzultácii sme sa rozhodli snímky spájať

In [ ]:
#vytvorenie potrebných priečinkov
input_folder = "BU_mercedes_stars"
output_folder = "BU_mercedes_combined"

os.makedirs(output_folder, exist_ok=True)

#funkcia na získanie timestampu
def parse_timestamp(filename):
    parts = filename.split("_")
    timestamp = parts[:6]
    return datetime(*map(int, timestamp))

In [ ]:
image_groups = {}
#iterácia pre každý obrázok
for filename in os.listdir(input_folder):
    if filename.endswith(".png"):
        #získanie časových údajov
        timestamp = parse_timestamp(filename)
        minute = timestamp.minute
        #roztriedenie na pol hodinové intervalové skupiny
        half_hour_segment = '00' if minute < 30 else '30'
        hour_key = timestamp.strftime(f"%Y_%m_%d_%H_{half_hour_segment}")
        if hour_key not in image_groups:
            image_groups[hour_key] = []
        image_groups[hour_key].append(os.path.join(input_folder, filename))

#pre každú skpinu a všetky snímky v nej
for hour_key, filenames in image_groups.items():
    images = [Image.open(filename).convert("RGB") for filename in filenames]
    #rozmery obrázkov
    width, height = images[0].size
    combined_image = np.zeros((height, width, 3), dtype=np.float32)
    #každí obrázok prevedený na array a pridaný do jedného
    for img in images:
        img_array = np.array(img, dtype=np.float32)
        combined_image += img_array / len(images)
    #vytvorenie spojeného snímku
    combined_image = np.clip(combined_image, 0, 255).astype(np.uint8)
    combined_image = Image.fromarray(combined_image)

    combined_image.save(os.path.join(output_folder, hour_key + "_mercedes.png"))